In [2]:
import pandas as pd

df = pd.read_excel("rekam_jejak_cypher_data.xlsx")
df

,Nomor Ketentuan,Jenis Bank,Mencabut,Dicabut,Mengubah,Diubah,Kodifikasi Ketentuan,Jenis Ketentuan,Judul Ketentuan,Tanggal Ketentuan
0,11/1/PBI/2009,Bank Umum Konvensional,2/27/PBI/2000,NaN,NaN,NaN,Bank Umum Konvensional,PBI,Bank Umum,22 Januari 2009
1,11/1/PBI/2009,Bank Umum Konvensional,23/3/BPPP/1990,NaN,NaN,NaN,Bank Umum Konvensional,PBI,Bank Umum,22 Januari 2009
2,11/1/PBI/2009,Bank Umum Konvensional,NaN,NaN,NaN,13/27/PBI/2011,Bank Umum Konvensional,PBI,Bank Umum,22 Januari 2009
3,13/27/PBI/2011,Bank Umum Konvensional,NaN,NaN,11/1/PBI/2009,NaN,Bank Umum Konvensional,PBI,Perubahan Atas Peraturan Bank Indonesia Nomor ...,28 Desember 2011
4,11/1/PBI/2009,Bank Umum Konvensional,2/27/PBI/2000,NaN,NaN,NaN,Bank Umum Konvensional,PBI,Bank Umum,22 Januari 2009
...,...,...,...,...,...,...,...,...,...,...
13639,43/SEOJK.03/2017,Bank Umum Konvensional,8/27/DPNP/2006,NaN,NaN,NaN,Unit Usaha Syariah,SEOJK,Prinsip Kehatihatian dan Laporan Dalam Rangka ...,19 Juli 2017
13640,43/SEOJK.03/2017,Bank Umum Syariah,8/27/DPNP/2006,NaN,NaN,NaN,Unit Usaha Syariah,SEOJK,Prinsip Kehatihatian dan Laporan Dalam Rangka ...,19 Juli 2017
13641,11/SEOJK.03/2017,Unit Usaha Syariah,14/20/DPNP,NaN,NaN,NaN,Unit Usaha Syariah,SEOJK,Prinsip Kehatihatian Bagi Bank Umum yang Melak...,17 Maret 2017
13642,11/SEOJK.03/2017,Bank Umum Konvensional,14/20/DPNP,NaN,NaN,NaN,Unit Usaha Syariah,SEOJK,Prinsip Kehatihatian Bagi Bank Umum yang Melak...,17 Maret 2017


In [3]:
import time

month_translation = {
    'Januari': 'January', 'Februari': 'February', 'Maret': 'March', 'April': 'April', 'Mei': 'May', 'Juni': 'June', 
    'Juli': 'July', 'Agustus': 'August', 'September': 'September', 'Oktober': 'October', 'November': 'November', 'Desember': 'December'
}

def translate_month(date_str):
    for indonesian, english in month_translation.items():
        date_str = date_str.replace(indonesian, english)
    return date_str

df['Tanggal Ketentuan'] = df['Tanggal Ketentuan'].apply(translate_month)
df['Tanggal Ketentuan'] = pd.to_datetime(df['Tanggal Ketentuan'], format='%d %B %Y')


In [5]:
df.head()

,Nomor Ketentuan,Jenis Bank,Mencabut,Dicabut,Mengubah,Diubah,Kodifikasi Ketentuan,Jenis Ketentuan,Judul Ketentuan,Tanggal Ketentuan
0,11/1/PBI/2009,Bank Umum Konvensional,2/27/PBI/2000,NaN,NaN,NaN,Bank Umum Konvensional,PBI,Bank Umum,2009-01-22
1,11/1/PBI/2009,Bank Umum Konvensional,23/3/BPPP/1990,NaN,NaN,NaN,Bank Umum Konvensional,PBI,Bank Umum,2009-01-22
2,11/1/PBI/2009,Bank Umum Konvensional,NaN,NaN,NaN,13/27/PBI/2011,Bank Umum Konvensional,PBI,Bank Umum,2009-01-22
3,13/27/PBI/2011,Bank Umum Konvensional,NaN,NaN,11/1/PBI/2009,NaN,Bank Umum Konvensional,PBI,Perubahan Atas Peraturan Bank Indonesia Nomor ...,2011-12-28
4,11/1/PBI/2009,Bank Umum Konvensional,2/27/PBI/2000,NaN,NaN,NaN,Bank Umum Konvensional,PBI,Bank Umum,2009-01-22


In [26]:
from py2neo import Graph, Node, Relationship

from langchain_community.graphs import Neo4jGraph

# Connect to the Neo4j database
graph = Neo4jGraph(url="bolt://108.137.122.156:7687", username="neo4j", password="crayongpt")

In [28]:
from py2neo import Graph, Node, Relationship

# Connect to the Neo4j database
graph = Graph("bolt://108.137.122.156:7687", name="neo4j", password="crayongpt")

In [29]:
# Create nodes
nodes = {}
for index, row in df.iterrows():
    nomor_ketentuan = row['Nomor Ketentuan']
    if nomor_ketentuan not in nodes:
        node = Node("Peraturan", id=nomor_ketentuan, nomor_ketentuan=nomor_ketentuan, 
                    jenis_bank=row['Jenis Bank'], jenis_ketentuan=row['Jenis Ketentuan'], judul=row['Judul Ketentuan'], 
                    tanggal_berlaku=row['Tanggal Ketentuan'])
        graph.merge(node, "Peraturan", "id")
        nodes[nomor_ketentuan] = node

# Function to create relationships
def create_relationship(source_id, target_id, relationship_type):
    if target_id not in nodes:
        node = Node("Peraturan", id=target_id, nomor_ketentuan=target_id)
        graph.merge(node, "Peraturan", "id")
        nodes[target_id] = node
    source_node = nodes[source_id]
    target_node = nodes[target_id]
    relationship = Relationship(source_node, relationship_type, target_node)
    graph.merge(relationship)

# Create relationships based on columns
for index, row in df.iterrows():
    source_id = row['Nomor Ketentuan']
    
    if pd.notna(row['Mencabut']):
        create_relationship(source_id, row['Mencabut'], "MENCABUT")
    
    if pd.notna(row['Dicabut']):
        create_relationship(source_id, row['Dicabut'], "DICABUT")
    
    if pd.notna(row['Mengubah']):
        create_relationship(source_id, row['Mengubah'], "MENGUBAH")
    
    if pd.notna(row['Diubah']):
        create_relationship(source_id, row['Diubah'], "DIUBAH")

Failed to write data to connection IPv4Address(('108.137.122.156', 7687)) (ResolvedIPv4Address(('108.137.122.156', 7687)))
Failed to write data to connection IPv4Address(('108.137.122.156', 7687)) (ResolvedIPv4Address(('108.137.122.156', 7687)))
Failed to write data to connection IPv4Address(('108.137.122.156', 7687)) (ResolvedIPv4Address(('108.137.122.156', 7687)))


In [34]:
len(graph.nodes)

476